In [ ]:
# install all libs and packages
!pip install pypdf
!pip install cohere
!pip install langchain
!pip install python-dotenv
!pip install openai
!pip install tiktoken
!pip install unstructured
!pip install pdf2image
!pip install pdfminer
!pip install kaleido
!pip install python-multipart
!pip install chromadb
!pip install -U langchain-openai

In [ ]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv, find_dotenv
import os

load_dotenv(find_dotenv())

False

In [ ]:
loader = PyPDFLoader("/content/doc_1_updated.pdf")

In [ ]:
data = loader.load()
print (f'You have {len(data)} document(s) in your data')
print (f'There are {len(data[0].page_content)} characters in your sample document')
print (f'Here is a sample: {data[0].page_content[:200]}')

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(data)

print (f'Now you have {len(texts)} documents')

Now you have 505 documents


In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings


OPENAI_API_KEY = os.getenv('OPENAI_API_KEY', 'API key')
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [ ]:
persist_directory = '/content/'

In [ ]:
# load it into Chroma
vectorstore = Chroma.from_documents(texts, embeddings, persist_directory=persist_directory)

In [ ]:
vectorstore.persist()

In [ ]:
vectorstore=None
vectorstore=Chroma(persist_directory=persist_directory, embedding_function=embeddings)
vectorstore.get()

In [ ]:
query = "Что такое типовая учебная программа?"
docs = vectorstore.similarity_search(query)
# Here's an example of the first document that was returned
for doc in docs:
    print (f"{doc.page_content}\n")

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain

llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
chain = load_qa_chain(llm, chain_type="stuff")

In [ ]:
query = "Какие есть требования к текстам учебников?"
docs = vectorstore.similarity_search(query)

In [ ]:
chain.run(input_documents=docs, question=query)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'Требования к текстам учебников включают:\n\n1) Соответствие государственным общеобязательным стандартам образования для различных уровней образования (дошкольного, начального, основного, среднего, технического, профессионального, послесреднего).\n\n2) Представление информации, которая соответствует типовой учебной программе, типовому учебному плану и учебнику.\n\n3) Включение заданий и упражнений, которые включают три этапа работы с текстом: предтекстовый, текстовый и послетекстовый.\n\n4) Взаимосвязь грамматических структур с лексическим материалом.\n\n5) Системность и логичность подачи грамматического материала.\n\n6) Соответствие учебных материалов интересам целевой аудитории.\n\nЭто основные требования к текстам учебников, но могут быть и другие специфические требования в зависимости от конкретного контекста и образовательной системы.'

In [ ]:
query = "оқулықтардың құрылымы нені қамтиды?"
docs = vectorstore.similarity_search(query)
chain.run(input_documents=docs, question=query)

'Оқулықтардың құрылымы автормен дидактикалық жəне əдістемелік тұрғысынан өңделген жəне жүйеленген сөздік жəне иллюстрациялық оқу материалын қамтиды.'

In [ ]:
query = "какие есть дексрипторы у критерия 'Введение к разделу' "
docs = vectorstore.similarity_search(query)
chain.run(input_documents=docs, question=query)

'Дескрипторы критерия "Введение к разделу" включают:\n\n1) Описаны цели обучения, понятные и мотивирующие учащихся.\n2) Подводит обучающихся к основной теме.\n3) Раскрывает последовательность содержания учебного материала, поясняет особенности изучения раздела.'

In [ ]:
query = "какие есть дексрипторы у критерия 'Соответствие языка изложения возрасту и познавательному уровню' ?"
docs = vectorstore.similarity_search(query)
chain.run(input_documents=docs, question=query)

'Дескрипторы для критерия "Соответствие языка изложения возрасту и познавательному уровню" включают:\n\n1) Не менее 90% текста понятно при первом чтении.\n2) Тексты увлекательные и интересные, способствуют обогащению словарного запаса.\n3) Оптимальность структуры и объема предложений в текстах.'

In [ ]:
query = "'Бөлімге кіріспе' критерийдің қандай дескрипторлар бар?"
docs = vectorstore.similarity_search(query)
chain.run(input_documents=docs, question=query)

'Бөлімге кіріспе критерийінің дескрипторлары:\n\n1) Түсінікті жəне оқушыларды ынталандыратын оқыту мақсаттары сипатталған.\n2) Білім алушыларды негізгі тақырыпты түсінуге бағыттайды.\n3) Оқу материалы мазмұнының реттілігін ашады, бөлімді оқу ерекшеліктерін түсіндіреді.\n\nЕгер сізде басқа сұраулар болса, смайлықтарды пайдаланыңыз. 😊'